

![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/streamlit_notebooks/healthcare_jsl/NER_CHEMPROT_CLINICAL.ipynb)




# **Detect chemical compounds and genes**

To run this yourself, you will need to upload your license keys to the notebook. Just Run The Cell Below in order to do that. Also You can open the file explorer on the left side of the screen and upload `license_keys.json` to the folder that opens.
Otherwise, you can look at the example outputs at the bottom of the notebook.



## 1. Colab Setup

Import license keys

In [ ]:
# Install the johnsnowlabs library to access Spark-OCR and Spark-NLP for Healthcare, Finance, and Legal.
! pip install johnsnowlabs

In [ ]:
from google.colab import files
print("Please Upload your John Snow Labs License using the button below")
license_keys = files.upload()

In [ ]:
from johnsnowlabs import *

# After uploading your license run this to install all licensed Python Wheels and pre-download Jars the Spark Session JVM
# Make sure to restart your notebook afterwards for changes to take effect

jsl.install()

## Start Session

In [ ]:
from johnsnowlabs import *
# Automatically load license data and start a session with all jars user has access to
spark = jsl.start()

In [ ]:
spark

## 2. Select the NER model and construct the pipeline

Select the NER model

For more details: https://github.com/JohnSnowLabs/spark-nlp-models#pretrained-models---spark-nlp-for-healthcare

In [ ]:
MODEL_NAME = "ner_chemprot_clinical"

Create the pipeline

In [ ]:
document_assembler = nlp.DocumentAssembler() \
    .setInputCol('text')\
    .setOutputCol('document')

sentence_detector = nlp.SentenceDetector() \
    .setInputCols(['document'])\
    .setOutputCol('sentence')

tokenizer = nlp.Tokenizer()\
    .setInputCols(['sentence']) \
    .setOutputCol('token')

word_embeddings = WordEmbeddingsModel.pretrained('embeddings_clinical', 'en', 'clinical/models') \
    .setInputCols(['sentence', 'token']) \
    .setOutputCol('embeddings')

clinical_ner = medical.NerModel.pretrained(MODEL_NAME, "en", "clinical/models") \
.setInputCols(["sentence", "token", "embeddings"])\
.setOutputCol("ner")

ner_converter = nlp.NerConverter()\
    .setInputCols(['sentence', 'token', 'ner']) \
    .setOutputCol('ner_chunk')

nlp_pipeline = Pipeline(stages=[document_assembler, 
                                sentence_detector,
                                tokenizer,
                                word_embeddings,
                                clinical_ner,
                                ner_converter])

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
ner_chemprot_clinical download started this may take some time.
[OK!]


## 3. Create example inputs

In [ ]:
# Enter examples as strings in this array
input_list = [
    """Emerging role of epidermal growth factor receptor inhibition in therapy for advanced malignancy: focus on NSCLC. Combination chemotherapy regimens have emerged as the standard approach in advanced non-small-cell lung cancer. Meta-analyses have demonstrated a 2-month increase in median survival after platinum-based therapy vs. best supportive care, and an absolute 10% improvement in the 1-year survival rate. Just as importantly, cytotoxic therapy has produced benefits in symptom control and quality of life. Newer agents, including the taxanes, vinorelbine, gemcitabine, and irinotecan, have expanded our therapeutic options in the treatment of advanced non-small-cell lung cancer. Despite their contributions, we have reached a therapeutic plateau, with response rates seldom exceeding 30-40% in cooperative group studies and 1-year survival rates stable between 30% and 40%. It is doubtful that substituting one agent for another in various combinations will lead to any further improvement in these rates. The thrust of current research has focused on targeted therapy, and epidermal growth factor receptor inhibition is one of the most promising clinical strategies. Epidermal growth factor receptor inhibitors currently under investigation include the small molecules gefitinib (Iressa, ZD1839) and erlotinib (Tarceva, OSI-774), as well as monoclonal antibodies such as cetuximab (IMC-225, Erbitux). Agents that have only begun to undergo clinical evaluation include CI-1033, an irreversible pan-erbB tyrosine kinase inhibitor, and PKI166 and GW572016, both examples of dual kinase inhibitors (inhibiting epidermal growth factor receptor and Her2). Preclinical models have demonstrated synergy for all these agents in combination with either chemotherapy or radiotherapy, leading to great enthusiasm regarding their ultimate contribution to lung cancer therapy. However, serious clinical challenges persist. These include the identification of the optimal dose(s); the proper integration of these agents into popular, established cytotoxic regimens; and the selection of the optimal setting(s) in which to test these compounds. Both gefitinib and erlotinib have shown clinical activity in pretreated, advanced non-small-cell lung cancer, but placebo-controlled randomized Phase III studies evaluating gefitinib in combination with standard cytotoxic therapy, to our chagrin, have failed to demonstrate a survival advantage compared with chemotherapy alone.""",
]

## 4. Use the pipeline to create outputs

In [ ]:
from pyspark.sql.types import StringType, IntegerType

df = spark.createDataFrame(input_list,StringType()).toDF('text')
result = nlp_pipeline.fit(df).transform(df)

result.select(F.explode(F.arrays_zip(result.ner_chunk.result, 
                                     result.ner_chunk.metadata)).alias("cols"))\
      .select(F.expr("cols['0']").alias("chunk"),
              F.expr("cols['1']['entity']").alias("ner_label")).show(30, truncate=False)

+--------------------------------+---------+
|chunk                           |ner_label|
+--------------------------------+---------+
|epidermal growth factor receptor|GENE-Y   |
|taxanes                         |CHEMICAL |
|vinorelbine                     |CHEMICAL |
|gemcitabine                     |CHEMICAL |
|irinotecan                      |CHEMICAL |
|epidermal growth factor receptor|GENE-Y   |
|Epidermal growth factor receptor|GENE-Y   |
|gefitinib                       |CHEMICAL |
|Iressa                          |CHEMICAL |
|ZD1839                          |CHEMICAL |
|erlotinib                       |CHEMICAL |
|Tarceva                         |CHEMICAL |
|OSI-774                         |CHEMICAL |
|cetuximab                       |CHEMICAL |
|IMC-225                         |CHEMICAL |
|Erbitux                         |CHEMICAL |
|CI-1033                         |CHEMICAL |
|tyrosine kinase                 |GENE-N   |
|PKI166                          |CHEMICAL |
|GW572016 

## 5. Visualize results

In [ ]:
from sparknlp_display import NerVisualizer

NerVisualizer().display(
    result = result.collect()[0],
    label_col = 'ner_chunk',
    document_col = 'document'
)